In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# load ascii text and covert to lowercase
path_to_file = tf.keras.utils.get_file('donald-tweets-clean.txt', 'https://github.com/Varun-S-Nair/Trump-Tweet-Generator/raw/master/RNNBackend/donald-tweets-clean.txt')
raw_text = open(path_to_file, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

294912/291367 [==============================] - 0s 0us/step


In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  291204
Total Vocab:  58


In [4]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  291104


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
temp = 1.0
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Lambda(lambda x: x / temp))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
# define the checkpoint
filepath="weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [8]:
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
291104/291104 [==============================] - 1340s 5ms/step - loss: 2.8479 - accuracy: 0.2053

Epoch 00001: loss improved from inf to 2.84794, saving model to weights-01-2.8479.hdf5
Epoch 2/20
291104/291104 [==============================] - 1352s 5ms/step - loss: 2.5330 - accuracy: 0.2816

Epoch 00002: loss improved from 2.84794 to 2.53302, saving model to weights-02-2.5330.hdf5
Epoch 3/20
291104/291104 [==============================] - 1341s 5ms/step - loss: 2.3129 - accuracy: 0.3484

Epoch 00003: loss improved from 2.53302 to 2.31286, saving model to weights-03-2.3129.hdf5
Epoch 4/20
291104/291104 [==============================] - 1313s 5ms/step - loss: 2.1701 - accuracy: 0.3886

Epoch 00004: loss improved from 2.31286 to 2.17008, saving model to weights-04-2.1701.hdf5
Epoch 5/20
291104/291104 [==============================] - 1325s 5ms/step - loss: 2.0709 - accuracy: 0.4164

Epoch 00005: loss improved from 2.17008 to 2.07089, saving model to weights-05-2.0709.hdf5

In [0]:
# load the network weights
filename = "weights-17-1.6736.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" e funeral of a wonderful and truly respected woman, phyllis s!
just returned from pensacola, florida "
 and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the debate and the

KeyboardInterrupt: ignored